In [1]:
%qtconsole

In [3]:
import pandas as pd
import os
import re
import romkan
import unicodedata

In [4]:
# get irregular words from ../data/irregular/irregular.csv and create a dict to avoid those words

In [5]:
vocab = pd.read_csv('../data/created/JMdict_e.csv', sep='\t')
kanji = pd.read_csv('../data/created/kanjidic2.csv')

In [11]:
vocab.tail()

,vocab,pronunciation,pos,meaning
141825,四つの力,よっつのちから,noun (common) (futsuumeishi),fundamental interactions
141826,等価原理,とうかげんり,noun (common) (futsuumeishi),equivalence principle
141827,目で追う,めでおう,"expressions (phrases, clauses, etc.)",to follow with one's gaze
141828,怪奇現象,かいきげんしょう,noun (common) (futsuumeishi),"haunting, unnatural phenomenon"
141829,強化学習,きょうかがくしゅう,noun (common) (futsuumeishi),reinforcement learning


In [10]:
PronunciationTypeCounter(vocab)

NameError: name 'ons' is not defined

In [8]:
# Run once
del vocab['pos']
del vocab['meaning']
del kanji['Unnamed: 0']
vocab.pronunciation = vocab.pronunciation.map(lambda x: romkan.to_hiragana(romkan.to_roma(x)))
kanji.on = kanji.on.map(lambda x: re.findall("""[\u3040-\u309f]+""", romkan.to_hiragana(romkan.to_roma(x))))
##############################
#kanji.kun = kanji.kun.map(lambda x: list(set(map(lambda y: y.split('.')[0],re.findall("""[\u3040-\u309f.]+""", x))))) # THIS THREW AWAY GOOD INFO
##############################
kanji['kun1'] = kanji.kun.map(lambda x: list(set(map(lambda y: y.split('.')[0],re.findall("""[\u3040-\u309f.]+""", x)))))
kanji['kun2'] = kanji.kun.map(lambda x: list(set(map(lambda y: ''.join(y.split('.')),re.findall("""[\u3040-\u309f.]+""", x)))))
# I should also include the kun without splitting somehow... aka shi.ku = shi and shiku
kanji.nanori = kanji.nanori.map(lambda x: re.findall("""[\u3040-\u309f]+""", x))
kanji.meaning = kanji.meaning.map(lambda x: re.findall("""[a-zA-Z-]+ *[a-zA-Z-]+""", x))
#char_ser = pd.Series(kanji.on.values,index=kanji.character).to_dict()
kanji['onkun'] = kanji.kun1 + kanji.kun2 + kanji.on
#kanji.onkun.map(lambda x: list(set(x))) ########## NOW MAKE SURE ALL ARE SET TO LISTS SO THERE ARE NO DUPLICATES
char_ser = pd.Series(kanji.onkun.values,index=kanji.character).to_dict()
kanji.head()

,character,freq,meaning,on,kun,nanori,kun1,kun2,onkun
0,亜,1509.0,"[Asia, rank next, come after, -ous]",[あ],['つ.ぐ'],"[や, つぎ, つぐ]",[つ],[つぐ],"[つ, つぐ, あ]"
1,唖,NaN,"[mute, dumb]","[あ, あく]",['おし'],[],[おし],[おし],"[おし, おし, あ, あく]"
2,娃,NaN,[beautiful],"[あ, あい, わ]",['うつく.しい'],[い],[うつく],[うつくしい],"[うつく, うつくしい, あ, あい, わ]"
3,阿,1126.0,"[Africa, flatter, fawn upon, corner, nook, rec...","[あ, お]","['おもね.る', 'くま']","[ほとり, あず, あわ, おか, きた, な]","[くま, おもね]","[くま, おもねる]","[くま, おもね, くま, おもねる, あ, お]"
4,哀,1715.0,"[pathetic, grief, sorrow, pathos, pity, sympat...",[あい],"['あわ.れ', 'あわ.れむ', 'かな.しい']",[],"[あわ, かな]","[かなしい, あわれ, あわれむ]","[あわ, かな, かなしい, あわれ, あわれむ, あい]"


In [9]:
#vocab.vocab = vocab.vocab.map(lambda x: romkan.to_hiragana(romkan.to_roma(x)))
#vocab.head()

#char_ser['亜']

#print(jlpt[jlpt.Kanji.str.contains('風')])
#print(char_ser['風'])

In [10]:
irregular = pd.read_csv('../data/irregular/irregular.csv')
jlpt = pd.read_csv('../data/vocab/jlptvocab.csv')
del irregular['URL']

In [11]:
irregular.head()

,Link
0,愛斯蘭
1,銅
2,欠伸
3,飛魚
4,麻布


In [12]:
jlpt = jlpt[jlpt.Kanji.isnull() == 0]
jlpt = jlpt[jlpt.Hiragana.isnull() == 0]
jlpt = jlpt[jlpt.English.isnull() == 0]
jlpt.tail()

,Kanji,Hiragana,English
8334,忘れ物,わすれもの,lost article
8335,笑う,わらう,"to laugh,to smile"
8336,割合,わりあい,"rate,ratio,percentage"
8337,割れる,われる,to break
8353,消しゴム,けしゴム,"eraser, rubber"


In [13]:
def unvoice(char):
    if not ('\u3041' <= char <= '\u3096') or len(char) > 1 or not unicodedata.decomposition(char):
        return char
    return unicodedata.normalize('NFD', char)[0]

In [14]:
### DONT RUN ###
#def check_word(word,cur,pron,pastprons,pastpron):
#    if cur > len(word):
#        return pron
#    try:
#        prons = set(char_ser[word[cur]])
#    except:
#        print(word, ' is not in the dictionary')
#    
#    past_prons.append('')
####

In [15]:
# if prons[-1] == ん: prons = [prons,prons[:-1]]
# if 
def recurs(kanji, word, pron_list = [], done = False):
    if len(pron_list) == len(kanji) or len(word) == 0: # check for 々 and repeat past syllable (unsounded)
        return pron_list, False
    cur = kanji[len(pron_list)]
    if re.match("""[\u3040-\u30ff]""", cur) != None: #should we romkan to hiragana?? ALSO HYPHEN
        if cur != word[0]:
            return pron_list, False
        prons = [cur]
    elif cur == '々':
        if unvoice(pron_list[-1]) == unvoice(word[:len(pron_list[-1])]): # Sou Zou -- didn't catch!!
            pron_list.append(word[:len(pron_list[-1])])
            return pron_list, len(kanji) == len(pron_list)
        else:
            return None, False
    else:
        prons = list(set(char_ser[cur])) # [char_ser[cur]] ###.sort(key=len, reverse=True)
    for pron in prons:        # GET RID OF THIS?
        if pron[-1] == 'ん'and pron != 'ん':
            if isinstance(prons,str):
                prons = [pron, pron[:-1]]
            elif pron[:-1] not in prons:
                prons.append(pron[:-1])
    for pron in prons:
        wordmatch = word[:len(pron)] ###-1
        wordmatch = unvoice(wordmatch[0]) + wordmatch[1:]
        words = [wordmatch, word[:len(pron)]] ### -1
        #print(kanji, word, pron_list, words)
        if pron in words:
            pron_list.append(word[:len(pron)])
            word = word[len(pron):]
            if len(pron_list) == len(kanji) and len(word) == 0:
                return pron_list, True
            new_pron_list, done = recurs(kanji,word,pron_list,done)
            if done:
                return new_pron_list, done
            word = pron_list.pop() + word
    return None, False

# update with unvoice the first syllable of word


# PRONUNCIATION DOES NOT MATCH ON or KUN YOMIKATA... MOSTLY DROPPING LAST N??? -------------

In [16]:
list(set(char_ser['騒'])) #中腹
#'boy'[1:]
#recurs('相変わらず','あいかわらず',[])
#print(recurs('指輪','ゆびわ',[]))
#char_ser['一']#.sort(key=len, reverse=True)    # EITHER INCLUDE NANORI OR FIX THE ABOVE

['そう', 'さわぐ', 'うれい', 'さわ', 'さわがしい']

In [17]:
true_cnt = 0
false_cnt = 0
false_dict = {}
true_dict = {}

for _,line in jlpt.iterrows():
    kanji = line.Kanji
    word = line.Hiragana
    #print(kanji,word) ###
    a,b = recurs(kanji,word,[])
    #print(a,b) ###
    if a == None:
        false_cnt += 1
        false_dict[kanji] = word
    else:
        true_cnt += 1
        true_dict[kanji] = a
print(true_cnt, false_cnt) #~8% are irregular...
print(false_dict)

6707 578
{'天皇': 'てんのう', '物語': 'ものがたり', '明後日': 'あさって', '景色': 'けしき', '八百屋': 'やおや', '実行': 'じっこう', '何方': 'どなた', '此の': 'この', '質素': 'しっそ', '如何にも': 'いかにも', '生年月日': 'せいねんがっぴ', '相応しい': 'ふさわしい', '圧迫': 'あっぱく', '若干': 'じゃっかん', '平均': 'ならし', '雑貨': 'ざっか', '打合せ': 'うちあわせ', '藍褸': 'ぼろ', '其れ程': 'それほど', '浮気': 'うわき', '面皰': 'にきび', '達する': 'たっする', '日当': 'にっとう', '清々しい': 'すがすがしい', '一日': 'ついたち', '可成': 'かなり', '微笑む': 'ほほえむ', '余程': 'よっぽど', '紅葉': 'もみじ', '実習': 'じっしゅう', '列島': 'れっとう', '二十日': 'はつか', '其れに': 'それに', '愈々': 'いよいよ', '滅多に': 'めったに', '出生': 'しゅっしょう', '煌々と': 'こうこうと', '其れでは': 'それでは', '塵芥': 'ごみ', '目盛': 'めもり', '説得': 'せっとく', '如何して': 'どうして', '掃除': 'そうじする', '疾っくに': 'とっくに', '如何しても': 'どうしても', '詰る': 'なじる', '匹敵': 'ひってき', '楽む': 'たのしむ', '失調': 'しっちょう', '何時': 'いつ', '灰': 'あく', '出産': 'しゅっさん', '其れで': 'それで', '何時か': 'いつか', '真面目': 'まじめ', '出品': 'しゅっぴん', '発掘': 'はっくつ', '巨': 'こ', '眼鏡': 'めがね', '仮令': 'たとえ', '目覚し': 'めざまし', '気質': 'かたぎ', '一切': 'いっさい', '発想': 'はっそう', '年中': 'ねんじゅう', '百科事典': 'ひゃっかじてん', '脱する': 'だっする', '何故なら': 'なぜなら', '日光': 'にっこう', '

In [18]:
true_dict['言う']

['い', 'う']

In [19]:
true_dict.keys() & false_dict.keys()

{'一定',
 '一日',
 '七日',
 '三日月',
 '上る',
 '下る',
 '乾',
 '今日',
 '他人',
 '何々',
 '例',
 '前',
 '勉強',
 '十',
 '否',
 '唯',
 '商人',
 '国境',
 '土産',
 '女子',
 '姉妹',
 '少女',
 '平均',
 '年月',
 '弟',
 '悪口',
 '所々',
 '掃除',
 '擦る',
 '散歩',
 '明後日',
 '暖かい',
 '月日',
 '木綿',
 '末',
 '本当',
 '東',
 '梅雨',
 '梯子',
 '洒落',
 '流行',
 '火傷',
 '灰',
 '眼鏡',
 '私',
 '突く',
 '紅葉',
 '練習',
 '背負う',
 '言う',
 '誰',
 '身体',
 '近々',
 '途中',
 '鈍い',
 '館'}

In [ ]:
false_dict['言う']

In [ ]:
#jlpt[jlpt.Kanji.str.contains('亜')]
#jlpt[jlpt.Kanji.isnull()]
print(recurs('十一人','じゅういちにん',[]))
print(recurs('一人','ひとり',[]))
#print(jlpt)

In [ ]:
char_on = pd.Series(kanji.on.values,index=kanji.character).to_dict()
char_kun = pd.Series(kanji.kun.values,index=kanji.character).to_dict()

In [ ]:
#jlpt[jlpt.Kanji.str.contains('亜')]
jlpt.head()

In [ ]:
### THIS WAS A PREVIOUS TEST
word = '漢数字'
pron = 'かんすうじ'
for ji in word:
    pron_list = re.findall("""[\u3040-\u309f]+""", char_ser[ji])
    pron_list.sort(key=len, reverse=True)
    print(pron_list)
    for pr in pron_list:
        if pron[0:len(pr)] == pr: # re match beginning of string
            print(ji, pr)
            pron = pron[len(pr):]
    #re.match()
    
#next match up each kanji to on or kun for exploratory analysis

#Maybe if word is only kanji - first check on list, if word is mixture - first check kun?

# Also put in checks for hirgana matching ()